In [1]:
#include <cassert>
#include <iostream>
#include <string>
#include <fstream>

In [2]:
template <class T, class A = std::allocator<T>>
struct Tuple : private std::vector<T, A>
{
  // fixed sized list
  using Tuple::vector::vector;
  using Tuple::vector::operator=;
  using Tuple::vector::get_allocator;
  using Tuple::vector::at;
  using Tuple::vector::front;
  using Tuple::vector::back;
  using Tuple::vector::data;
  using Tuple::vector::begin;
  using Tuple::vector::cbegin;
  using Tuple::vector::end;
  using Tuple::vector::cend;
  using Tuple::vector::empty;
  using Tuple::vector::size;
  using Tuple::vector::operator[];
};

In [3]:
class Indices {
   // Abstraction for a finite sequence of indices,
   // where each 'index' is a multi-index i.e. it 
   // is itself a `dim`-sized sequence of integers,
   // say e.g. ((0,0), (0,1), (0,2), (1,1), ... ),
   // for `dim=2`. The integers of a current
   // multi-index are held in a buffer. This class
   // serves to modify the buffer with its `next`
   // method to simulate sequence traversal,
   // i.e. it is a generator.
   public:
      // By the sequence being exhausted we mean
      // that it has been taken beyond its last element.
      // When the index represents the last element of
      // the sequence, the returned value is false.
      virtual bool is_exhausted() = 0;
      // Connect some internal pointer to the buffer with indices.
      virtual void use_buff(unsigned int* const) = 0;
      // Number of indices (lenght of buffer)
      unsigned int dim;
      // Move the index one step forward in the sequence.
      virtual void next() = 0; 
      // Change the index back to the first element of the sequence.
      virtual void reset() = 0;
      // Return a printable representation of the current index.
      virtual std::string to_str() = 0;
      // Traverse the sequence, printing subsequent indices.
      void print(unsigned limit = -1) {
         assert(index);
         unsigned count = 0;
         while(!is_exhausted()) {
            if (count == limit)
               break;
            std::cout << to_str() << std::endl;
            next();
            ++count;
         }
         reset();
      }
};

In [4]:
class IndicesTriangle : public Indices {
   // Each index in the sequence is an increasing sequence.
   // Notice that this subclass also implements the
   // degenerated cases of zero- and one-element sequences,
   // as well as the zero-dimensional sequence.
   protected:
      // Pointer to a buffer with actual integers.
      unsigned int* index = NULL;
      // Lower bound (inclusive) of all values of all indices.
      unsigned int lower;
      // Upper bound (noninclusive) of all values of all indices.
      unsigned int upper;
      // Is the index-sequence not strictly increasing;
      // i.e. is e.g. (0, 1, 1, 2) in the sequence?
      bool with_diag;
      // Was the whole sequence exhausted?
      bool exhausted;
   public:
      IndicesTriangle (unsigned int lower_ = 0 ,
                       unsigned int upper_ = 0,
                       unsigned int dim_ = 0,
                       bool with_diag_ = true) :
         lower(lower_),
         upper(upper_),
         with_diag(with_diag_),
         exhausted(lower == upper) {
         dim = dim_;
         assert(upper >= lower);
         assert(with_diag | (upper - lower >= dim));
      }
      bool is_exhausted() {
         return exhausted;
      }
      void use_buff(unsigned int* const buff) {
         index = buff;
      }
      void next() {
         if (upper == lower) { return ;}
         // F-style traversal
         else if (dim > 0) {
            bool shift_right = 1;
            for(unsigned int position = 0; position < dim - 1; ++position) {
               shift_right = (index[position] + 1) / (index[position + 1] + with_diag);
               if (shift_right) {
                  index[position] = lower + !with_diag * position;
               } else {
                  ++index[position];
                  break;
               }
            }
            if (shift_right) {
               ++index[dim - 1];
               exhausted = index[dim - 1] / upper;
            }
         }
         else {
            exhausted = true;
         }
      }
      void next_() {
         if (upper == lower) { return ;}
         // C-style traversal
         else if (dim > 0) {
            unsigned int position = dim - 1;
            bool shift_left = (index[position] + 1) / upper;
            while (shift_left) {
               if (position == 0) {
                  exhausted = true;
                  return;
               }
               --position;
               shift_left = (index[position] + !with_diag) / index[position + 1];
            }
            ++index[position];
            for (unsigned int i = position + 1; i < dim; ++i) {
               index[i] = index[i - 1] + !with_diag;
            }  
         }
         else {
            exhausted = true;
         }
      }
      void reset() {
         if (upper > lower) {
            for(unsigned int position = 0; position < dim; ++position)
               index[position] = lower + !with_diag * position;
            exhausted = false;
         }
      }
      std::string to_str() {
         std::stringstream ss;
         if (dim > 0 & upper > lower) {
            ss << "(";
            for (unsigned int i = 0; i < dim - 1; ++i)
                ss << index[i] << ", ";
            ss << index[dim - 1] << ")";
         }
         else {
            ss << "";
         }
         return ss.str();
      }
};

In [5]:
Tuple<IndicesTriangle> my_triangles(3);

In [6]:
const unsigned int dim = 5;

In [7]:
unsigned int dummy_index[dim];

In [9]:
my_triangles[0] = IndicesTriangle(0, 3, dim);

In [10]:
my_triangles[0].use_buff(dummy_index)

In [11]:
my_triangles[0].print();

(0, 0, 0, 0, 0)
(0, 0, 0, 0, 1)
(0, 0, 0, 1, 1)
(0, 0, 1, 1, 1)
(0, 1, 1, 1, 1)
(1, 1, 1, 1, 1)
(0, 0, 0, 0, 2)
(0, 0, 0, 1, 2)
(0, 0, 1, 1, 2)
(0, 1, 1, 1, 2)
(1, 1, 1, 1, 2)
(0, 0, 0, 2, 2)
(0, 0, 1, 2, 2)
(0, 1, 1, 2, 2)
(1, 1, 1, 2, 2)
(0, 0, 2, 2, 2)
(0, 1, 2, 2, 2)
(1, 1, 2, 2, 2)
(0, 2, 2, 2, 2)
(1, 2, 2, 2, 2)
(2, 2, 2, 2, 2)
